# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. 


In [1]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier
import detoxify
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

## Подготовка

In [2]:
df = pd.read_csv('D:/YP_DS/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Все ячейки заполнены, имеют корректный тип данных и не имеют пропусков

In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df.loc[:159199]

In [5]:
df.toxic.value_counts()

0    143020
1     16180
Name: toxic, dtype: int64

Дропнем лишний столбец `Unnamed: 0`, а также избавимся от последний 92-ух строк, для того чтобы кол-во строк в нашем были кратны "батчу" в предстоящей операции создания эмбеддингов

В датасете имеется также сильный дисбаланс классов, который в дальнейшем будет сбалансирован с помощью `SMOTE`

In [6]:
model_class, tokenizer_class, pretrained_weights = (
    ppb.AutoModel, ppb.AutoTokenizer, "unitary/toxic-bert")

Данная модель это лишь хорошо затюненная модель `DistilBERT-a`, предназначенная для классификации токсичных комментариев\
`DistilBERT` обрабатывает предложения и передает извлеченную им информацию в следующую модель. `DistilBERT` представляет собой уменьшенную версию `BERT'а`, разработанную и выложенную в отрытый доступ группой разработчиков **_HuggingFace_**. Она быстрее и легче своего старшего собрата, но при этом вполне сравнима в результативности.

In [7]:
%%time
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

#Токенизируем наш текст и заменяем каждый токен на уникальый идентификатор 
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True))

#Дополняем каждую строку нулями до тех пор , пока не достигнем максимальной длины одной из строк
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array(
    [i + [0]*(max_len - len(i)) for i in tokenized.values])

# Создаём маску "внимания", для того чтобы игнорировать добавленные нули
attention_mask = np.where(padded != 0, 1, 0)

CPU times: total: 32.9 s
Wall time: 33.4 s


In [8]:
#Инициализируем модель
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
#Подключаем видеокарту, Чтобы не умереть в ожидании))
cuda0 = torch.device('cuda:0')
torch.cuda.empty_cache()

In [10]:
%%time
device = torch.device("cuda")
model.cuda()

batch_size = 100
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
#         embeddings.append(batch_embeddings[0].cpu().numpy())


100%|██████████| 1592/1592 [38:03<00:00,  1.43s/it]

CPU times: total: 1h 15min 6s
Wall time: 38min 3s


In [11]:
#Соберём все эмбеддинги в матрицу признаков вызовом функции concatenate():
features = np.concatenate(embeddings)
features = pd.DataFrame(features)

target = df['toxic']

In [12]:
#Сплитим две выборки , где тестовая выборка будет равна 10%
features_train,features_test,target_train,target_test = train_test_split(features,target,test_size = .1,random_state=42)

## Обучение моделей

In [31]:
time_list = []

In [32]:
def teach_model(model, parameters, features, target,time_list,pipeline = False):
    
    if pipeline == True:
        imba_pipeline = make_pipeline(SMOTE(random_state=42,n_jobs=-1), 
                                  model)   
        grid = GridSearchCV(imba_pipeline, parameters, n_jobs=-1,
                            scoring='f1',cv=3)
    else:
        grid = GridSearchCV(model, parameters, n_jobs=-1,
                            scoring='f1',cv=3)
    grid.fit(features, target)
    print(grid.best_score_)
    print(grid.best_params_)
    
    time_list.append([grid.cv_results_['mean_fit_time'][0], grid.cv_results_[
                     'mean_score_time'][0], grid.best_score_])

    return grid.best_estimator_

### Логистическая регрессия без балансировки классов

In [33]:
%%time
params = {"C":[1,5,15]}
model = LogisticRegression()
lr_no_balance = teach_model(model, params, features_train,target_train,time_list)


0.9468863243168991
{'C': 1}
CPU times: total: 7.11 s
Wall time: 41.9 s


### Логистическая регрессия c апсемплингом и без кросс-валидации

In [34]:
train_features_upsample,train_target_upsample = SMOTE(random_state=42,n_jobs=-1).fit_sample(features_train,target_train)

In [35]:
lr_no_cv = LogisticRegression()
lr_no_cv.fit(train_features_upsample,train_target_upsample)
f1_score(train_target_upsample,lr_no_cv.predict(train_features_upsample))

0.9873290114457643

### Логистическая регрессия c апсемплингом как часть кросс-валидации

In [36]:
%%time
params = {"logisticregression__C":[1,5,15]}
model = LogisticRegression()
lr_balanced = teach_model(model, params, features_train,target_train,time_list,pipeline=True)


0.9272562955843869
{'logisticregression__C': 5}
CPU times: total: 42.7 s
Wall time: 1min 35s


### Решающее дерево

In [37]:
%%time
params = {"decisiontreeclassifier__max_depth":[3,4,5]}
model = DecisionTreeClassifier()
best_dt = teach_model(model, params, features_train,target_train,time_list,pipeline=True)


0.9071130848310981
{'decisiontreeclassifier__max_depth': 4}
CPU times: total: 1min 50s
Wall time: 2min 52s


### CatBoostClassifier

In [38]:
%%time
params = {'catboostclassifier__iterations': [100,400], 'catboostclassifier__learning_rate': [.1, .5]}
model = CatBoostClassifier()
best_cb = teach_model(model, params, features_train,target_train,time_list,pipeline=True)


0:	learn: 0.4778249	total: 163ms	remaining: 1m 4s
1:	learn: 0.3329020	total: 309ms	remaining: 1m 1s
2:	learn: 0.2469203	total: 451ms	remaining: 59.7s
3:	learn: 0.1851667	total: 592ms	remaining: 58.6s
4:	learn: 0.1393362	total: 747ms	remaining: 59s
5:	learn: 0.1128339	total: 898ms	remaining: 58.9s
6:	learn: 0.0959367	total: 1.02s	remaining: 57.6s
7:	learn: 0.0844041	total: 1.17s	remaining: 57.3s
8:	learn: 0.0756527	total: 1.3s	remaining: 56.5s
9:	learn: 0.0692454	total: 1.45s	remaining: 56.4s
10:	learn: 0.0643520	total: 1.59s	remaining: 56.4s
11:	learn: 0.0608724	total: 1.73s	remaining: 56s
12:	learn: 0.0584756	total: 1.87s	remaining: 55.8s
13:	learn: 0.0566212	total: 1.99s	remaining: 54.8s
14:	learn: 0.0548670	total: 2.12s	remaining: 54.4s
15:	learn: 0.0534705	total: 2.25s	remaining: 53.9s
16:	learn: 0.0523072	total: 2.36s	remaining: 53.2s
17:	learn: 0.0512952	total: 2.49s	remaining: 52.9s
18:	learn: 0.0504936	total: 2.63s	remaining: 52.7s
19:	learn: 0.0497986	total: 2.77s	remaining: 5

In [40]:
pd.DataFrame(columns=['mean_learning_time (sec)', 'mean_fit_time (sec)','f1'],
             index=['LogisticRegression non balanced','LogisticRegression balanced', 'DecisionTree','CatBoost'],
             data=time_list)

,mean_learning_time (sec),mean_fit_time (sec),f1
LogisticRegression non balanced,33.366134,0.348039,0.946886
LogisticRegression balanced,74.743368,0.382085,0.927256
DecisionTree,57.116592,0.053345,0.907113
CatBoost,112.032214,1.375483,0.936774


Наилучшая метрика `f1` на кросс-валидации была достигнута моделью `CatBoostClassifier`, поэтому для дальнейшнего тестирования выберем именно эту модель

## Тестирование

In [41]:
predict = lr_no_balance.predict(features_test)

In [42]:
f1_score(target_test,predict)

0.9508495145631067

In [43]:
accuracy_score(target_test,predict)

0.9898241206030151

Имеем хорошую метрику `f1 > 0.75`, которая соответствует условию задачи\
Также имеем отличный показатель точности для данной модели

## Проверка модели на адекватность

In [44]:
dummy = DummyClassifier()
dummy.fit(features_train, target_train)
dummy_predict = dummy.predict(features_test)

print('f1:',f1_score(target_test, dummy_predict))

f1: 0.0


Метрика `f1` на тестировочной выборке наилучшей модели сильно выше , Чем метрика константной модели.\
Модель проходит проверку на адекватность